In [1]:
# Gerekli kütüphaneleri projemize dahil ediyoruz.
import pandas as pd
import re
import numpy as np
import math

In [ ]:
# Class oranlarımızı hesaplıyoruz
# ÖRN N = 4 Yes = 3 No = 1
# P(Yes) = 3/4, P(No) = 1/4
def labelPredictions(Y):
    # Key-value şeklinde tanımlamak için boş bir list tanımlıyoruz
    labels = {}
    
    # Toplam example sayımızı öğreniyoruz
    total = len(Y)
    
    # Her sınıf stününü gezip sayılarını alıyoruz
    for label in Y:            
        if label in labels:
            labels[label] += 1
        else:
            labels[label] = 1
    
    # Her label'in oranını hesaplıyoruz.
    for i in labels:
        val = labels[i]

        labels[i] = val / total;

    return labels

# Parametre olarak alınınan string cümlenin içindeki kelimelerin
# harflerini küçülterek alıyoruz.
def split_words(sentence):
    words = re.sub("[^\w]", " ",  sentence).split()
    words = list(map(lambda x:x.lower(),words))

    return words

# Parametre olarak alınan cümlenin içindeki kelimeri
# benzersiz bir şekilde alıp yine aynı kelimenin
# cümle içinde kaç defa kullanıldığı bilgisi ile birlikte alıyoruz.
def split_words_unique(sentence):
    words = split_words(sentence)

    _words = {}

    for w in words:
        if w not in _words:
            _words[w] = 1
        else:
            _words[w] += 1

    return _words

# Vocabulary değerini hesaplıyoruz
def calculateVocabulary(X):
    amount = 0
    stack = []

    for sentence in X:
        words = split_words(sentence)
        

        for w in words:
            if w not in stack:
                stack.append(w)
                amount += 1

    return amount

# Kelime sayısını hesaplıyoruz
def determineWordsCount(X):
    count = 0

    for sentence in X:
        words = split_words(sentence)

        for w in words:
            count += 1

    return count

# Sınıf içindeki kelime sayısını alıyoruz
# ÖRN: P(Chinese|Yes) = 5 , P(Tokyo|Yes) = 0
def getWordCountInClass(payload,word,c):
    df = dataFrameForClass(payload,c)

    sentences = df[payload['f_text']]

    count = 0

    for sentence in sentences:
        words = split_words(sentence)

        for w in words:
            if w == word:
                count += 1

    return count

# Belirli bir sınıfa ait data frame alıyoruz.
# ÖRN: Sadece 'Yes' sınıfına ait text ve label data frame dönecektir.
def dataFrameForClass(payload,c):
    return payload['X'].loc[payload['X'][payload['f_label']] == c]

# Spesifik olarak belirlnen sınıfa ait kelime sayısın verir.
def getWordsCount(payload,c):
    df = dataFrameForClass(payload,c)

    return determineWordsCount(df[payload['f_text']])

# Mödelimizi ayarlıyoruz ve gerekli parametreleri bir
# list olarak geri alıyoruz
def fit(X,Y,f_text = 'sentence',f_label = 'label'):

    payload = {};
    
    # Toplam sınıf sayımızı alıyoruz
    payload['classes'] = set(Y)
    # Class oranlarımızı alıyoruz
    # P(Sport) = 2 / 5 P(Not Sport) = 3 / 5
    payload['predictions'] = labelPredictions(Y)
    # Vocabulary değerini alıyoruz
    payload['vocabulary'] = calculateVocabulary(X[f_text])
    
    # Diğer veri modellerini kullanmak için
    # veri listimize kaydediyoruz
    payload['X'] = X
    payload['Y'] = Y
    payload['f_text'] = f_text
    payload['f_label'] = f_label

    return payload

# Tahminleme işlemlerini gerçekleştiriyoruz
def predict(payload,text):
    # Cümle içindeki benzersiz kelimeleri alıyoruz
    words = split_words_unique(text)
    
    m_estimate = {}
    
    # M-Estimate hesaplaması yapıyoruz
    for c in payload['classes']:
        n = getWordsCount(payload,c)

        m_estimate[c] = {}

        for word in words:
            force = words[word]

            n_c = getWordCountInClass(payload,word,c)

            # P(d|c) = (n_c + 1) / (n + vocabulary)
            _estimate = (n_c + 1) / (n + payload['vocabulary'])
            
            _estimate = math.pow(_estimate,force)

            m_estimate[c][word] = _estimate

    tags = {}
    
    # Hesaplanan herbir class değerlerine göre hesaplama yapıp bunu
    # 'tags' objesine aktarıyoruz
    # ÖRN: P(d|Yes) = 0,0003 --- P(d|No) = 0,0001
    for c in payload['predictions']:
        p = payload['predictions'][c]

        m = np.prod(list(m_estimate[c].values()))

        final = m * p

        tags[c] = final
        
    # print(tags)
    
        
    # Son olarak bulunan etiketlerin arasında
    # en yüksek değere sahip etiketi geriye dönderiyoruz.
    # Bu şekilde label değerimiz ortaya çıkmış oluyor
    return max(tags,key= lambda x: tags[x])
            
        

In [3]:
df = pd.read_csv('processed_train.csv')

payload = fit(df,df['label'])

label = predict(payload,"안녕하세요")

print(label)

{'E68': 1.1208127826648813e-07, 'E32': 1.2843289489702288e-07, 'E50': 1.0175598832855088e-07, 'E67': 1.1156239928820677e-07, 'E33': 1.0122764633229616e-07, 'E24': 1.305255408069496e-07, 'E17': 1.0838175327588178e-07, 'E12': 1.0384250494354629e-07, 'E27': 1.1491585828097071e-07, 'E42': 1.0877522273402433e-07, 'E52': 1.028945325659383e-07, 'E55': 1.0679436846959813e-07, 'E35': 1.1268621701092414e-07, 'E10': 1.2705644276776926e-07, 'E31': 1.2167509939122202e-07, 'E28': 1.0622047385711334e-07, 'E36': 1.0259571720555082e-07, 'E25': 1.0078476590391041e-07, 'E65': 9.66004903150907e-08, 'E58': 1.1195632748509463e-07, 'E26': 9.533509202042995e-08, 'E16': 9.967312959701058e-08, 'E59': 1.1006914442882218e-07, 'E37': 1.5315655759048925e-07, 'E38': 1.0593239329851243e-07, 'E44': 1.1607042177791072e-07, 'E40': 1.0651184432344817e-07, 'E30': 1.1364879528708967e-07, 'E18': 1.1466581412552542e-07, 'E60': 1.1546648094439027e-07, 'E13': 1.2683430014302295e-07, 'E34': 1.0785601545062305e-07, 'E29': 1.0419

In [13]:
label = predict(payload,"그렇게 말하니까 마음 아프네")

print(label)

{'E1': 3.0752234765410644e-18, 'E2': 1.0487684045734952e-17, 'E3': 3.62003720736254e-18, 'E5': 5.097271576057071e-18, 'E4': 1.0430828573329226e-17, 'E6': 2.379072591221906e-18}
E2


In [4]:
df_test_small = pd.read_csv('processed_test.csv', error_bad_lines=False, engine="python", encoding='utf-8-sig')

/home/mhkim/anaconda3/envs/rex_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:

correct = 0;
incorrect = 0;

incMap = {}
incMapTwo = {}

for index, row in df_test_small.iterrows():
    print(str(index))
    label = row['label']
    sentence = row['sentence']
    est = predict(payload,sentence)
    if(est == label):
        correct+=1
    else:
        incorrect+=1
        if(label in incMap):
            incMap[label] = incMap[label] + 1
        else:
            incMap[label] = 1    
        if(est in incMapTwo):
            incMapTwo[est] = incMapTwo[est] + 1
        else:
            incMapTwo[est] = 1
        # print("Estimated LABEL: " + getReadableClass(est))
        # print("Correct LABEL: " + getReadableClass(label))
        # print("WRONG : " + sentence)

for elem in incMap:
    print(elem + " : " + str(incMap[elem]*100/incorrect))

print("========================")
    
for elemTwo in incMapTwo:
    print(elemTwo + " : " + str(incMapTwo[elemTwo]*100/incorrect))
        
print("Accuracy : " + str(correct/(correct+incorrect)))

0
{'E68': 1.64637662003396e-18, 'E32': 1.4974739300123283e-17, 'E50': 4.805107284820232e-19, 'E67': 5.855451607022883e-19, 'E33': 1.807673808587963e-18, 'E24': 3.4074008937283977e-17, 'E17': 1.6024328109755085e-18, 'E12': 7.629795946449875e-18, 'E27': 2.6675350839749395e-18, 'E42': 6.678595493444241e-19, 'E52': 1.4452839415304292e-18, 'E55': 1.3287772320421486e-18, 'E35': 2.183728712743998e-18, 'E10': 2.6200434849098565e-18, 'E31': 4.082031302323454e-18, 'E28': 2.9118951805762986e-18, 'E36': 1.4644343686800064e-18, 'E25': 3.1782306784610323e-18, 'E65': 5.066565280438841e-19, 'E58': 1.763416899049605e-18, 'E26': 3.9894666000481627e-19, 'E16': 1.3439455483065635e-18, 'E59': 2.5461323813814137e-18, 'E37': 1.2557264350778259e-17, 'E38': 1.906032698327048e-18, 'E44': 2.0915456459578698e-18, 'E40': 3.4433991143061276e-18, 'E30': 4.3342875329303784e-18, 'E18': 2.79175967725641e-18, 'E60': 1.2054841653131892e-18, 'E13': 7.160574115291401e-18, 'E34': 9.784595691127184e-18, 'E29': 1.449986794426

KeyboardInterrupt: 